קודים של הקדמה

In [26]:

import pandas as pd
import geopandas as gpd


In [27]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [28]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [29]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [51]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df

להעלות את הטבלה של גורי

In [30]:
col=['sector', 'geometry', 'ele_stu', 'mid_stu', 'high_stu']

In [31]:
shcl=up_load_shp(r'W:\Data\Forecast\תחומי אחריות\יצירת מידע\בתי ספר\education.shp')[col] #לסנן רק את השכבות שצריך בלי גנים

העלת שכבת טאז

In [32]:
col=['Taz_num','Taz_name','main_secto','Muni_Heb','zonetype','geometry']

In [33]:
taz=up_load_shp(r'W:\Data\Forecast\Tools\forecast_git\create_forecast_basic\current\background_files\TAZ_V4_230518_Published.shp')[col]

להחבר לגיאוגרפיה ולהוריד אזורים שמחוץ לתחום שלנו

In [34]:
taz_without_palestinian=taz.loc[taz['main_secto']!='Palestinian']

In [35]:
shcl_with_taz=gpd.sjoin(taz_without_palestinian,shcl).drop(columns='geometry')

לסכום לפי אזור תנועה ומגזר

לשנות את השמות עמודות לפי איך שאני מודלים צריכים

In [36]:
old_col=['ele_stu','mid_stu','high_stu']

In [37]:
shcl_with_taz_arab=shcl_with_taz.loc[shcl_with_taz['sector']==1].pivot_table(index='Taz_num',aggfunc=sum)[old_col]

In [38]:
new_col=['ARA1','ARA2','ARA3']

In [39]:
shcl_with_taz_arab.columns=new_col

In [40]:
shcl_with_taz_hardi=shcl_with_taz.loc[shcl_with_taz['sector']==2].pivot_table(index='Taz_num',aggfunc=sum)[old_col]

In [41]:
new_col=['UOA1','UOA2','UOA3']

In [42]:
shcl_with_taz_hardi.columns=new_col

In [43]:
shcl_with_taz_Jewish=shcl_with_taz.loc[shcl_with_taz['sector']==3].pivot_table(index='Taz_num',aggfunc=sum)[old_col]

In [44]:
new_col=['SEA1','SEA2','SEA3']

In [45]:
shcl_with_taz_Jewish.columns=new_col

לבדוק אל מול נתוני למ"ס מה המצב של המידע שגורי יצר

כדי לעשות את ההשוואה צריך לצרף את הנתונים לטבלת אזורי התנועה ולייצא


In [46]:
taz.merge(shcl_with_taz_Jewish.reset_index(), on='Taz_num', how='left') \
   .merge(shcl_with_taz_hardi.reset_index(), on='Taz_num', how='left') \
   .merge(shcl_with_taz_arab.reset_index(), on='Taz_num', how='left') \
   .to_excel('delet.xlsx')


לייצר טבלה עם כמות תלמידים לפי רשות

In [60]:
software_folder_location=r'W:\Data\Forecast\Tools\forecast_git\create_forecast_basic\current'

In [61]:
muni_JTMT=up_load_shp(r'{}\background_files\muni_under_JTMT_ITM.shp'.format(software_folder_location))


In [62]:
col=['CR_PNIM','Muni_Heb']
muni_JTMT=muni_JTMT[col]
muni_JTMT['CR_PNIM']=muni_JTMT['CR_PNIM'].astype(int)

In [63]:
# Define the directory path
directory_path = r'{}\background_files'.format(software_folder_location)

# # Load each DataFrame separately
df1 = up_load_df(directory_path, 'cbs_student_2020_by_muni_3')
df2 = up_load_df(directory_path, 'cbs_student_2020_by_muni_1')
df3 = up_load_df(directory_path, 'cbs_student_2020_by_muni_2')

# # Concatenate the DataFrames
student_gov_by_muni = pd.concat([df1, df2, df3])

In [64]:
col=[ 'סמל_יישוב',
 'תלמידים_בבתי_ספר_יסודיים_תש_ף_2019_20',
 'תלמידים_בחטיבות_ביניים_תש_ף_2019_20',
 'תלמידים_בבתי_ספר_תיכוניים_תש_ף_2019_20']

student_gov_by_muni=student_gov_by_muni[col]

In [65]:
student_gov_by_muni=student_gov_by_muni.merge(muni_JTMT,left_on='סמל_יישוב',right_on='CR_PNIM')
student_gov_by_muni=student_gov_by_muni.drop_duplicates(subset='CR_PNIM',keep='first')


In [66]:
col=[ 'תלמידים_בבתי_ספר_יסודיים_תש_ף_2019_20',
 'תלמידים_בחטיבות_ביניים_תש_ף_2019_20',
 'תלמידים_בבתי_ספר_תיכוניים_תש_ף_2019_20']

In [67]:
student_gov_by_muni['student_gov']=student_gov_by_muni[col].sum(axis=1)

In [68]:
student_gov_by_muni.to_excel('gov_muni_students_to_delet.xlsx')